# LSTM文本生成

In [44]:
# 依赖导入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# 文本数据加载(本地数据)
raw_data = open("./task2_data").read()
# 移除换行符
data = raw_data.replace("\n", "").replace("\r", "")
data

'Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": a

In [6]:
# 字母字符的去重处理
letters = list(set(data))
print(letters)
features = len(letters)
print(features)

['k', '"', 'f', 'z', 'x', ' ', 'I', 't', 'd', 'p', ':', 'y', 'm', 'a', 'L', '.', 'r', 'v', 's', 'h', 'o', 'b', 'n', ',', 'e', 'l', ')', 'c', 'u', 'g', 'i', '(', 'A']
33


In [11]:
# 建立字典
# 数字:字符字典：
int_to_char = {a: b for a, b in enumerate(letters)}
print(int_to_char)
# 字符:数字字典
char_to_int = {b: a for a, b in enumerate(letters)}
print(char_to_int)

{0: 'k', 1: '"', 2: 'f', 3: 'z', 4: 'x', 5: ' ', 6: 'I', 7: 't', 8: 'd', 9: 'p', 10: ':', 11: 'y', 12: 'm', 13: 'a', 14: 'L', 15: '.', 16: 'r', 17: 'v', 18: 's', 19: 'h', 20: 'o', 21: 'b', 22: 'n', 23: ',', 24: 'e', 25: 'l', 26: ')', 27: 'c', 28: 'u', 29: 'g', 30: 'i', 31: '(', 32: 'A'}
{'k': 0, '"': 1, 'f': 2, 'z': 3, 'x': 4, ' ': 5, 'I': 6, 't': 7, 'd': 8, 'p': 9, ':': 10, 'y': 11, 'm': 12, 'a': 13, 'L': 14, '.': 15, 'r': 16, 'v': 17, 's': 18, 'h': 19, 'o': 20, 'b': 21, 'n': 22, ',': 23, 'e': 24, 'l': 25, ')': 26, 'c': 27, 'u': 28, 'g': 29, 'i': 30, '(': 31, 'A': 32}


In [13]:
# time_step
time_step = 30

In [14]:
#滑动窗口提取数据
def extract_data(data, slide):    
    x = []
    y = []    
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])        
    return x,y

In [15]:
#字符到数字的批量转化
def char_to_int_Data(x,y, char_to_int):    
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])    
    return x_to_int, y_to_int

In [16]:
#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):    
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0], char_Data[1], char_to_int)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(Input), slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output

In [21]:
# 字符串处理
x, y = data_preprocessing(data, time_step, features, char_to_int)

In [26]:
print(x.shape, len(y))

(11160, 30, 33) 11160


In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1, random_state=0)

In [30]:
print(x.shape, x_train.shape, x_test.shape)

(11160, 30, 33) (10044, 30, 33) (1116, 30, 33)


In [32]:
# 输出结果的格式转化
y_train_category = to_categorical(y_train, features)

In [35]:
# 建立LSTM模型
lstm_model = Sequential()
# LSTM层
lstm_model.add(LSTM(units=30, input_shape=(30, 33), activation="relu"))
# 输出层
lstm_model.add(Dense(units=features, activation="softmax"))
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30)                7680      
_________________________________________________________________
dense_1 (Dense)              (None, 33)                1023      
Total params: 8,703
Trainable params: 8,703
Non-trainable params: 0
_________________________________________________________________


In [36]:
# 核心参数配置
lstm_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [74]:
# 模型训练
lstm_model.fit(x_train, y_train_category, batch_size=1000, epochs=150)

Epoch 1/150
11/11 [==============================] - 1s 54ms/step - loss: 0.0443 - accuracy: 0.9976
Epoch 2/150
11/11 [==============================] - 1s 55ms/step - loss: 0.0415 - accuracy: 0.9973
Epoch 3/150
11/11 [==============================] - 1s 54ms/step - loss: 0.0396 - accuracy: 0.9983
Epoch 4/150
11/11 [==============================] - 1s 53ms/step - loss: 0.0393 - accuracy: 0.9993
Epoch 5/150
11/11 [==============================] - 1s 53ms/step - loss: 0.0380 - accuracy: 0.9977
Epoch 6/150
11/11 [==============================] - 1s 53ms/step - loss: 0.0362 - accuracy: 0.9994
Epoch 7/150
11/11 [==============================] - 1s 53ms/step - loss: 0.0359 - accuracy: 0.9990
Epoch 8/150
11/11 [==============================] - 1s 52ms/step - loss: 0.0347 - accuracy: 1.0000
Epoch 9/150
11/11 [==============================] - 1s 55ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 10/150
11/11 [==============================] - 1s 53ms/step - loss: 0.0311 - accuracy: 0.9997

11/11 [==============================] - 1s 52ms/step - loss: 0.8740 - accuracy: 0.7096
Epoch 83/150
11/11 [==============================] - 1s 53ms/step - loss: 0.8304 - accuracy: 0.7144
Epoch 84/150
11/11 [==============================] - 1s 53ms/step - loss: 0.7719 - accuracy: 0.7426
Epoch 85/150
11/11 [==============================] - 1s 52ms/step - loss: 0.6867 - accuracy: 0.7728
Epoch 86/150
11/11 [==============================] - 1s 52ms/step - loss: 0.6308 - accuracy: 0.7985
Epoch 87/150
11/11 [==============================] - 1s 53ms/step - loss: 0.5697 - accuracy: 0.8203
Epoch 88/150
11/11 [==============================] - 1s 53ms/step - loss: 0.5591 - accuracy: 0.8217
Epoch 89/150
11/11 [==============================] - 1s 53ms/step - loss: 0.4828 - accuracy: 0.8531
Epoch 90/150
11/11 [==============================] - 1s 55ms/step - loss: 0.4224 - accuracy: 0.8696
Epoch 91/150
11/11 [==============================] - 1s 53ms/step - loss: 0.3943 - accuracy: 0.8789
Epo

In [75]:
# 训练数据的预测
y_train_predict = np.argmax(lstm_model.predict(x_train), axis=-1)
y_train_predict

array([24, 24,  5, ..., 24, 22,  8])

In [76]:
y_train_predict_char = [int_to_char[char] for char in y_train_predict]
print(y_train_predict_char)

['e', 'e', ' ', ' ', 'i', ' ', 'i', 'g', 'c', 's', 'a', 'e', 'a', ' ', 'e', ' ', 'e', 'e', 'o', 'k', 'g', 'l', 'l', 't', 'n', 'A', ' ', 'n', 's', 'i', 'a', ' ', 'l', 'g', 'b', 's', 'e', 'e', 'c', 'o', 't', 'i', 'i', ' ', 'a', 'n', 'e', 'r', 'e', ' ', 's', ' ', 'i', 'a', ',', ' ', 'n', 'l', 'e', 'm', 'i', 'i', 'f', 'l', 'f', 's', 'p', 'L', 'o', ' ', ' ', 'e', 's', 'n', 'f', 'b', ' ', 'g', ' ', 'i', 'i', 'o', 'e', 'e', ' ', 'i', 'u', 'g', 'h', 'n', 't', 'e', 'i', 'd', ':', ' ', ' ', 'e', ' ', 'i', 'i', 'i', 'I', 'l', 'c', 'l', 'd', 's', 'n', 'i', 'x', 'v', 'x', 'v', ' ', 'm', 'c', 's', 'a', 'v', ' ', 'c', 's', 'a', 'n', ' ', 'a', 'h', 'u', 'A', 's', ' ', 'e', 'a', 'n', ' ', 'o', ' ', 'e', 'd', 'e', 'c', 'n', '(', 'k', 't', 'u', 'd', 'k', 'i', 't', 'u', ' ', 'n', 't', 't', ' ', ' ', 's', 'd', 'i', 't', 'n', 'l', 'd', 'I', 'e', 'e', 'e', 'm', 'n', ' ', 'i', 'n', 'n', ' ', 'r', 'm', 'g', 's', 'x', ' ', 'a', 'a', 'm', 'c', ' ', 's', 'i', 'm', 'l', 't', 'i', ' ', ' ', ' ', 'd', ' ', 'd', ' ',

In [77]:
# 准确率
train_score = accuracy_score(y_train_predict, y_train)
train_score

1.0

In [78]:
# 测试数据的预测
y_test_predict = np.argmax(lstm_model.predict(x_test), axis=-1)

In [79]:
y_test_predict_char = [int_to_char[char] for char in y_test_predict]

In [80]:
test_score = accuracy_score(y_test, y_test_predict)

In [81]:
test_score

1.0

In [82]:
# 新字符串的预测
new_letters = "Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines"
x_new,y_new = data_preprocessing(new_letters, time_step, features, char_to_int)
print(x_new.shape, len(y_new))

(79, 30, 33) 79


In [83]:
y_new_predict = lstm_model.predict_classes(x_new)
print(y_new_predict)

[18 20 12 24  7 30 12 24 18  5 27 13 25 25 24  8  5 12 13 27 19 30 22 24
  5 30 22  7 24 25 25 30 29 24 22 27 24 23  5 30 18  5 30 22  7 24 25 25
 30 29 24 22 27 24  5  8 24 12 20 22 18  7 16 13  7 24  8  5 21 11  5 12
 13 27 19 30 22 24 18]


/Users/mac/workspaces/deep-learning-practise-venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [84]:
y_new_predict_char = [int_to_char[char] for char in y_new_predict]

In [85]:
print(y_new_predict_char)

['s', 'o', 'm', 'e', 't', 'i', 'm', 'e', 's', ' ', 'c', 'a', 'l', 'l', 'e', 'd', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', ',', ' ', 'i', 's', ' ', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', ' ', 'd', 'e', 'm', 'o', 'n', 's', 't', 'r', 'a', 't', 'e', 'd', ' ', 'b', 'y', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', 's']


In [86]:
for i in range(0, x_new.shape[0] - 30):
    print(new_letters[i:i+30],"--- predict new letter is>>>", y_new_predict_char[i])

Artificial intelligence (AI),  --- predict new letter is>>> s
rtificial intelligence (AI), s --- predict new letter is>>> o
tificial intelligence (AI), so --- predict new letter is>>> m
ificial intelligence (AI), som --- predict new letter is>>> e
ficial intelligence (AI), some --- predict new letter is>>> t
icial intelligence (AI), somet --- predict new letter is>>> i
cial intelligence (AI), someti --- predict new letter is>>> m
ial intelligence (AI), sometim --- predict new letter is>>> e
al intelligence (AI), sometime --- predict new letter is>>> s
l intelligence (AI), sometimes --- predict new letter is>>>  
 intelligence (AI), sometimes  --- predict new letter is>>> c
intelligence (AI), sometimes c --- predict new letter is>>> a
ntelligence (AI), sometimes ca --- predict new letter is>>> l
telligence (AI), sometimes cal --- predict new letter is>>> l
elligence (AI), sometimes call --- predict new letter is>>> e
lligence (AI), sometimes calle --- predict new letter is>>> d
ligence 